In [ ]:
!pip install ultralytics flask pyngrok opencv-python-headless --quiet
# Menginstall pustaka Python yang diperlukan seperti ultralytics, flask, pyngrok, dan opencv-python-headless menggunakan pip, dengan flag --quiet untuk menyembunyikan output detail.

from flask import Flask, request, render_template, string, send_file
# Mengimpor modul-modul tertentu (Flask, request, render_template, string, send_file) dari pustaka flask untuk membangun aplikasi web.

from ultralytics import YOLO
# Mengimpor kelas YOLO dari pustaka ultralytics untuk penggunaan model deteksi objek.

import cv2
# Mengimpor pustaka OpenCV (cv2) untuk pemrosesan gambar dan video.

import numpy as np
# Mengimpor pustaka NumPy sebagai np untuk operasi numerik dan array.

import os
# Mengimpor pustaka os untuk berinteraksi dengan sistem operasi, seperti mengelola file dan direktori.

from pyngrok import ngrok
# Mengimpor modul ngrok dari pustaka pyngrok untuk membuat terowongan aman ke aplikasi lokal.

In [ ]:
# Inisialisasi aplikasi Flask
app = Flask(__name__)

# Buat folder untuk menyimpan file sementara
os.makedirs('static/uploads', exist_ok=True)
os.makedirs('static/outputs', exist_ok=True)

# Muat model YOLOv8
model = YOLO('yolov8n.pt')

In [ ]:
# Template HTML untuk antarmuka web
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Pendeteksi Burung</title>
    <style>
        body { font-family: Arial, sans-serif; text-align: center; padding: 20px; }
        h1 { color: #333; }
        .container { max-width: 800px; margin: auto; }
        .result { margin-top: 20px; }
        img, video { max-width: 100%; height: auto; }
        input[type=file] { margin: 20px 0; }
        button { padding: 10px 20px; background-color: #4CAF50; color: white; border: none; cursor: pointer; }
        button:hover { background-color: #45a049; }
        .error { color: red; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Pendeteksi Burung dengan YOLOv8</h1>
        <p>Unggah gambar (.jpg, .png) atau video (.mp4) untuk mendeteksi burung.</p>
        <form method="post" enctype="multipart/form-data">
            <input type="file" name="file" accept="image/*,video/*" required>
            <br>
            <button type="submit">Unggah dan Deteksi</button>
        </form>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        {% if output_path %}
            <div class="result">
                <h3>Hasil Deteksi</h3>
                {% if is_image %}
                    <img src="{{ output_path }}" alt="Hasil Deteksi">
                {% else %}
                    <video controls>
                        <source src="{{ output_path }}" type="video/mp4">
                        Maaf, browser Anda tidak mendukung video.
                    </video>
                {% endif %}
                <br>
                <a href="{{ output_path }}" download>Unduh Hasil</a>
            </div>
        {% endif %}
    </div>
</body>
</html>
"""



In [ ]:
# Fungsi untuk memproses gambar
def process_image(image_path, output_path):
    # Membaca gambar dari path yang diberikan menggunakan OpenCV
    img = cv2.imread(image_path)
    # Mengonversi warna gambar dari BGR ke RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Melakukan deteksi objek pada gambar menggunakan model
    results = model(image_path)

    # Iterasi melalui setiap hasil deteksi
    for result in results:
        boxes = result.boxes
        # Iterasi melalui setiap kotak deteksi
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            # Mendapatkan nilai kepercayaan (confidence) dari deteksi
            confidence = box.conf[0].item()
            # Mendapatkan ID kelas dari deteksi
            class_id = int(box.cls[0].item())
            # Mendapatkan label berdasarkan ID kelas dari model
            label = model.names[class_id]

            # Memeriksa jika label adalah 'bird'
            if label == 'bird':
                # Menggambar kotak persegi panjang di sekitar objek yang terdeteksi
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Menambahkan teks label dan kepercayaan di atas kotak
                cv2.putText(img, f'{label} {confidence:.2f}', (x1, y1 - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Menyimpan gambar yang telah diproses ke path output dengan konversi kembali ke BGR
    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    # Mengembalikan path gambar yang telah diproses
    return output_path

In [ ]:
# Fungsi untuk memproses video
def process_video(video_path, output_path):
    # Membuka video dari path yang diberikan menggunakan OpenCV
    cap = cv2.VideoCapture(video_path)
    # Mendapatkan lebar frame dari video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # Mendapatkan tinggi frame dari video
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # Mendapatkan frame rate (FPS) dari video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Mengatur codec video untuk output (mp4v)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # Membuat objek VideoWriter untuk menyimpan video yang telah diproses
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Memproses video frame demi frame selama video masih terbuka
    while cap.isOpened():
        ret, frame = cap.read()
        # Menghentikan loop jika tidak ada frame lagi
        if not ret:
            break

        # Melakukan deteksi objek pada frame menggunakan model
        results = model(frame)

        # Iterasi melalui setiap hasil deteksi
        for r in results:
            for box in r.boxes:
                cls = int(box.cls[0])
                # Mendapatkan label berdasarkan ID kelas dari model
                label = model.names[cls]
                # Memeriksa jika label adalah 'bird'
                if label == 'bird':
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    # Menggambar kotak persegi panjang di sekitar objek yang terdeteksi
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    # Menambahkan teks label dan kepercayaan di atas kotak
                    cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Menulis frame yang telah diproses ke file output
        out.write(frame)

    # Melepaskan objek VideoCapture
    cap.release()
    # Melepaskan objek VideoWriter
    out.release()
    # Mengembalikan path video yang telah diproses
    return output_path

In [ ]:
# Route utama
@app.route('/', methods=['GET', 'POST'])
def index():
    # Mengecek metode request, jika POST maka memproses file yang diunggah
    if request.method == 'POST':
        # Mengecek apakah ada file yang diunggah
        if 'file' not in request.files:
            return render_template_string(HTML_TEMPLATE, error='Tidak ada file yang diunggah')

        # Mengambil file dari request
        file = request.files['file']
        # Mengecek apakah nama file kosong
        if file.filename == '':
            return render_template_string(HTML_TEMPLATE, error='Pilih file terlebih dahulu')

        # Mendapatkan nama file dan memeriksa ekstensi yang didukung
        filename = file.filename
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png', '.mp4')):
            return render_template_string(HTML_TEMPLATE, error='Format file tidak didukung. Gunakan .jpg, .png, atau .mp4')

        # Menyimpan path input file
        input_path = os.path.join('static/uploads', filename)
        file.save(input_path)

        # Menyusun nama dan path untuk file output
        output_filename = 'output_' + filename
        output_path = os.path.join('static/outputs', output_filename)

        # Menentukan apakah file adalah gambar
        is_image = filename.lower().endswith(('.jpg', '.jpeg', '.png'))
        try:
            # Memproses file sesuai tipe (gambar atau video)
            if is_image:
                process_image(input_path, output_path)
            else:
                process_video(input_path, output_path)
        except Exception as e:
            # Menangani error selama pemrosesan
            return render_template_string(HTML_TEMPLATE, error=f'Error saat memproses file: {str(e)}')

        # Mengembalikan template dengan path output
        return render_template_string(HTML_TEMPLATE, output_path='/' + output_path, is_image=is_image)

    # Mengembalikan template kosong untuk GET request
    return render_template_string(HTML_TEMPLATE)

# Route untuk file statis
@app.route('/static/<path:filename>')
def serve_static(filename):
    # Mengirim file statis dari direktori statis
    return send_file(os.path.join('static', filename))

# Setup ngrok dengan token Anda
!ngrok authtoken 2zKEqZK0KddGK3qLuEVOv2hxG1C_3M8KfiNoowgXg5FvqTKUC  # Token Anda
# Menghubungkan ngrok ke port 5000 dan mendapatkan URL publik
public_url = ngrok.connect(5000).public_url
print(f"🌐 Aplikasi berjalan di: {public_url}")

# Jalankan aplikasi Flask
if __name__ == '__main__':
    # Menjalankan aplikasi Flask di host dan port yang ditentukan
    app.run(host='0.0.0.0', port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Aplikasi berjalan di: https://b9bb-34-21-8-197.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an a

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

image 1/1 /content/static/uploads/burunnggg.jpg: 384x640 1 bird, 1 cat, 1 dining table, 279.7ms
Speed: 9.7ms preprocess, 279.7ms inference, 27.8ms postprocess per image at shape (1, 3, 384, 640)


Streaming output truncated to the last 5000 lines.
This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation 


image 1/1 /content/static/uploads/burunnggg.jpg: 384x640 1 bird, 1 cat, 1 dining table, 138.1ms
Speed: 4.0ms preprocess, 138.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Streaming output truncated to the last 5000 lines.
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed